In [198]:
import pandas as pd
import pubchempy as pcp
from tqdm.notebook import tqdm
import re
import pickle
import os
from termcolor import colored

In [199]:
polarity = "negative"
missing_name = "Undefined"
offenders = ("nan", "Untitled", "Oprea", "Opera", "AKO", "CHEMBL", "SR-", "SCHEM", "EU-", "MLS", "NSC", "ChemDiv", "ST0", "TimTec", "HMS", "BIM", "CB", "CCG-", "Cambridge", "SMR", "AB0", "BRD-", "NCG", "BDBM", "CBKinase", "BAS ", "ZINC", "GNF", "SQX", "CDS", "STK", "NCI", "TNP", "Boc-Tyr-OH", "PD", "UNM", "BSP", "CCRIS", "MFCD", "IDI", "ST5", "AC1", "WAY-", "KUC", "DTXSID", "MixCom", "CK-", "ASN ", "MMV", "SKI-", "VU", "SMSF", "Bio2", "REGID", "SDCC", "BCBc", "SMP", "TCMDC", "cid_", "BCP", "AST ", "SY0", "AM-", "IFLab")
save_pubchem_dict = True

In [200]:
c18_standards_df = pd.read_csv('./C18_standards_' + polarity + '.tsv', sep='\t', float_precision='round_trip')
inchikeys = c18_standards_df['inchi_key']

In [ ]:
if os.path.isfile('C18_standards_' + polarity + '_all-adducts_pubchem_dict.pkl'):
    
    print ("PubChem dictionary already exists in working directory - exiting")
    
else:

    inchikey_to_iupac = {}

    for key in tqdm(inchikeys):

        #print("Gathering data for " + key)

        if key in inchikey_to_iupac.keys():
            pass
        else:
            try:
                # Run to PubChem and look for InChI key
                cid = pcp.get_compounds(key, namespace='inchikey', as_dataframe=True, listkey_count=5).reset_index()['cid'].to_string(index=False)
                
                # Define compound properties
                if "\n" in cid:
                    cid = (cid.rstrip().split('\n'))[-1]
                else:
                    cid = cid
                compound = pcp.Compound.from_cid(cid)
                iupac_name = compound.iupac_name
                synonym_list = compound.synonyms
                
                # Check properties and assign to compound based on presence/absence in DB
                if not iupac_name:
                    iupac_name = missing_name
                if not synonym_list:
                    synonyms = missing_name
                else:
                    synonyms = synonym_list
                
                full_name = synonyms

            # If InChI key doesn't exist, create an undefined named that matches format
            except KeyError:
                
                print("Note: " + key + " is not found in PubChem!")

                full_name = missing_name
            
            inchikey_to_iupac[key] = (full_name)

    if save_pubchem_dict == True:
        
        filehandler = open('C18_standards_' + polarity + '_all-adducts_pubchem_dict.pkl', 'wb')
        pickle.dump(inchikey_to_iupac, filehandler)

In [202]:
# Sorts through the PubChem synonym list and chooses the best name

def fix_synonym_list(synonym_dict: dict) -> dict:

    for inchi_key in tqdm(synonym_dict):
            
        synonym_subset = synonym_dict[inchi_key]

        if isinstance(synonym_subset, str) == True:
            synonym_subset = [synonym_subset]

        synonym_subset = [x for x in synonym_subset if not x.startswith(offenders)]
        synonym_subset = list(filter(lambda x: not x.replace("-", "").replace(re.compile('^A-Z').pattern, "").isdigit(), synonym_subset))
        synonym_subset = list(filter(lambda x: not re.sub(r'\b[A-Z]{1}',"",x).isdigit(), synonym_subset))
        synonym_subset = [x for x in synonym_subset if not re.compile(r'\b[A-Z]{14}\b-\b[A-Z]{10}\b-\b[A-Z]{1}\b').search(x)]
        synonym_subset = [x for x in synonym_subset if not re.compile(r'^F\d{4}-\d{4}').search(x)]

        if not synonym_subset:
            top_synonym = missing_name
        else:
            top_synonym = synonym_subset[0]

        synonym_dict[inchi_key] = top_synonym

    return synonym_dict

In [ ]:
# Choose best synonym for each InChI key and write new dictionary (with single str as value instead of list)

with open('C18_standards_' + polarity + '_all-adducts_pubchem_dict.pkl', 'rb') as filehandler:

    pubchem_dictionary = pickle.load(filehandler)

    relabel_dict = fix_synonym_list(pubchem_dictionary)

In [204]:
# Fix 'label' column to remove internal "number: " and "adduct" features
 
c18_standards_df_relabel = c18_standards_df.copy()

c18_standards_df_relabel['label'] = c18_standards_df_relabel['label'].apply(lambda x: pd.Series(x.split(' ', 1)))[1].apply(lambda x: pd.Series(re.split(r' \[M', x)))[0]

In [205]:
# Create 'label' and 'name' dictionaries to fix those too

label_dict = c18_standards_df_relabel.set_index('inchi_key')['label'].to_dict()

name_dict = c18_standards_df_relabel.set_index('inchi_key')['name'].to_dict()

In [206]:
# Fixes the existing label and names column

def fix_label_and_name(input_dict:dict) -> dict:

    for inchi_key, value_string in tqdm(input_dict.items()):

        new_value_string = re.sub(r'\b[A-Z]{14}\b-\b[A-Z]{10}\b-\b[A-Z]{1}\b', 'Undefined', value_string)
        new_value_string = re.sub(r'^F\d{4}-\d{4}', 'Undefined', new_value_string)
        if new_value_string.startswith(offenders):
            new_value_string = 'Undefined'
        if new_value_string.replace("-", "").isdigit():
            new_value_string = 'Undefined'

        input_dict[inchi_key] = new_value_string
    
    return input_dict

In [ ]:
# Get all three name dictionaries in one place and check lengths

label_dict_update = fix_label_and_name(label_dict)
name_dict_update = fix_label_and_name(name_dict)
relabel_dict_update = relabel_dict

if len(label_dict_update.values()) == len(name_dict_update.values()) == len(relabel_dict_update.values()):
    print("All dictionaries have " + str(len(label_dict_update.keys())) + " keys. Good to go!")
else:
    print("Warning! Dictionaries don't have the same number of keys.")

In [208]:
# Manual curation is required for some edge cases

def choose_parsimonious_label(multi_name_df:pd.DataFrame) -> pd.DataFrame:
    
    multi_name_df_singled = multi_name_df.copy()
    multi_name_df_singled = multi_name_df_singled.replace("Undefined", ("Undefined"*50)) # Create artificially long Undefined string
    multi_name_df_singled['combo'] = multi_name_df_singled['relabel'] + ";;" + multi_name_df_singled['name'] + ";;" + multi_name_df_singled['label']
    multi_name_df_singled['combo'] = multi_name_df_singled['combo'].str.lower()
    multi_name_df_singled['new_label'] = multi_name_df_singled.combo.str.split(';;').apply(lambda x: min(x, key=len))

    if(polarity == "negative"):
        multi_name_df_singled['new_label'] = multi_name_df_singled['new_label'].str.replace("cape [m-h]-","caffeic acid phenethyl ester [m-h]-")
        multi_name_df_singled['new_label'] = multi_name_df_singled['new_label'].str.replace("2',4'-dihydroxyacetophenone [m-h]-","resacetophenone [m-h]-")
        multi_name_df_singled['new_label'] = multi_name_df_singled['new_label'].str.replace("egcg [m-h]-","epigallocatechin gallate [m-h]-")
        multi_name_df_singled['new_label'] = multi_name_df_singled['new_label'].str.replace("fmet [m-h]-","n-formyl-l-methionine [m-h]-")

    if(polarity == "positive"):
        multi_name_df_singled['new_label'] = multi_name_df_singled['new_label'].str.replace("c27h32fno2 [m+h]+","(2E)-1-[2-(4-fluorophenyl)-6-hydroxy-3-azabicyclo[4.4.0]dec-3-yl]-3-[4-(methyl ethyl)phenyl]prop-2-en-1-one [m+h]+")
        multi_name_df_singled['new_label'] = multi_name_df_singled['new_label'].str.replace("cobadex [m+h]+","cortisol [m+h]+")
        multi_name_df_singled['new_label'] = multi_name_df_singled['new_label'].str.replace("trp-gly [m+h]+","tryptophylglycine [m+h]+")
        multi_name_df_singled['new_label'] = multi_name_df_singled['new_label'].str.replace("c276-0086 [m+h]+","4-(4-hydroxy-3,5-dimethoxyphenyl)-6,7-dimethoxy-1,3,4-trihydroquinolin-2-one [m+h]+")
        multi_name_df_singled['new_label'] = multi_name_df_singled['new_label'].str.replace("antibiotic 9663 [m+h]+","ochratoxin a [m+h]+")
        multi_name_df_singled['new_label'] = multi_name_df_singled['new_label'].str.replace("sw219134-1 [m+h]+","silymarin [m+h]+")
        multi_name_df_singled['new_label'] = multi_name_df_singled['new_label'].str.replace("megxp0_000848 [m+h]+","haploperoside c [m+h]+")
        multi_name_df_singled['new_label'] = multi_name_df_singled['new_label'].str.replace("Œ± [m+h]+","alpha [m+h]+")
        multi_name_df_singled['new_label'] = multi_name_df_singled['new_label'].str.replace(".alpha. [m+h]+","alpha [m+h]+")
        multi_name_df_singled['new_label'] = multi_name_df_singled['new_label'].str.replace("egcg [m-h]-","epigallocatechin gallate [m-h]-")
        multi_name_df_singled['new_label'] = multi_name_df_singled['new_label'].str.replace("minocycline¬∑hcl [m-h]-","minocycline hydrochloride [m-h]-")
 
    return multi_name_df_singled

In [209]:
# Combine the three dictionaries and choose shortest string (likely to be the best)

combined_dictionaries = pd.DataFrame({'relabel':pd.Series(relabel_dict_update),'name':pd.Series(name_dict_update),'label':pd.Series(label_dict_update)})

new_label_df = choose_parsimonious_label(combined_dictionaries)

In [ ]:
# Insert new label into original file

c18_standards_df_updated = c18_standards_df.copy()
new_label_df_updated = new_label_df.copy()

new_label_df_updated['inchi_list'] = new_label_df_updated.index
new_label_df_updated = new_label_df_updated[['inchi_list','new_label']]
new_label_df_updated = new_label_df_updated.reset_index(drop=True)
new_label_df_updated

original_and_updated_merge = pd.merge(c18_standards_df_updated, new_label_df_updated, right_on='inchi_list', left_on='inchi_key', how='left')
original_and_updated_merge.insert(3, "new_label", original_and_updated_merge.pop('new_label'))
original_and_updated_merge = original_and_updated_merge.drop(original_and_updated_merge[['inchi_list','label']], axis=1)
original_and_updated_merge = original_and_updated_merge.rename(columns = {'new_label': 'label'})

if original_and_updated_merge.shape == c18_standards_df.shape:
    print("Original and updated dataframes have the same dimensions of " + str(original_and_updated_merge.shape) + ". Good to go!")
else:
    print("Warning! Original and updated dataframes have different dimensions.")


In [211]:
# Write new standards file with new label substituted

original_and_updated_merge.to_csv('C18_standards_' + polarity + '_all-adducts_renamed.tsv', sep='\t', index = False)

In [212]:
# Find all duplicated inchi_keys (with the same adduct)

original_and_updated_merge['inchi-adduct'] = original_and_updated_merge['inchi_key'].astype(str) + original_and_updated_merge['adduct']
duplicated_inchis = list(original_and_updated_merge[original_and_updated_merge['inchi-adduct'].duplicated()]['inchi-adduct'].drop_duplicates().values)

In [213]:
# Subset label-fixed standards file by only duplicated inchi keys

c18_standards_df_dups = original_and_updated_merge[original_and_updated_merge['inchi-adduct'].isin(duplicated_inchis)]

In [ ]:
# Calculate retention time deviations and print to file

duplicate_summary_stats = []

for inchi in tqdm(duplicated_inchis):

    subset_df = c18_standards_df_dups.loc[c18_standards_df_dups['inchi-adduct'] == inchi]
    inchi_key = subset_df['inchi_key'].tolist()[0]
    inchi_name = subset_df['label'].tolist()[0]
    inchi_adduct = subset_df['adduct'].tolist()[0]
    inchi_count = len(subset_df)
    inchi_mean = subset_df['rt_peak'].mean()
    inchi_std = subset_df['rt_peak'].std()
    inchi_files = subset_df['file_name'].tolist()

    duplicate_summary_stats.append([inchi, inchi_key, inchi_name, inchi_adduct, inchi_count, inchi_mean, inchi_std, inchi_files])

duplicate_summary_table = pd.DataFrame(duplicate_summary_stats)
duplicate_summary_table.rename(columns={0: 'inchi-adduct', 1: 'inchi_key', 2: 'label', 3: 'adduct', 4: 'duplications', 5: 'rt_peak_mean', 6: 'rt_peak_std', 7: 'file_names'}, inplace=True)
duplicate_summary_table.sort_values('rt_peak_std', ascending=False, inplace=True)

duplicate_summary_table.to_csv('C18_standards_' + polarity + '_all-adducts_duplicate_standards.tsv', sep='\t', index = False)

In [ ]:
# Choose which duplicated row will be retained with a decision tree

vetted_standards = pd.read_excel("top_two_per_polarity_from_drive.xlsx")
vetted_standards = vetted_standards[vetted_standards['polarity'] == polarity]

deduplicated_data = []
decision_dict = {}

for dupkey, adduct in tqdm(list(zip(duplicate_summary_table['inchi_key'], duplicate_summary_table['adduct']))):
    
    dupkey_df = original_and_updated_merge[(original_and_updated_merge['inchi_key'] == dupkey) & (original_and_updated_merge['adduct'] == adduct)]
    entries = dupkey_df.shape[0]

    print(dupkey + " " + adduct + " (" + str(entries) + " dups)")

    ## Decision tree:

    if vetted_standards['inchi_key'].str.contains(dupkey).any():

        top_hit = vetted_standards[(vetted_standards['inchi_key'] == dupkey) & (vetted_standards['adduct'] == adduct)]

        if 0 < top_hit.shape[0] <= 2:

            top_hit.sort_values('code_rank_without_msms', ascending = True)
            top_hit = top_hit.head(1)['file_name'].values[0]

            best_dupkey = dupkey_df[(dupkey_df['inchi_key'] == dupkey) & (dupkey_df['file_name'] == top_hit)]

            print(colored('Filtered: ', 'green') + 'Top hits table\n')
            deduplicated_data.append(best_dupkey)
            decision_dict[dupkey] = 'Top_hits_table'
            continue

        # else:
            
        #     print(colored('Warning: ', 'red') + 'Multiple compounds with identical inchikey')
        #     print(top_hit['label'].values)
        #     print('\n')
        #     continue

    if sum(dupkey_df['valid_msms']) == 1: ## Only single True
        
        best_dupkey = dupkey_df[dupkey_df['valid_msms'] == True]
        
        print(colored('Filtered: ', 'green') + 'Valid MSMS\n')
        deduplicated_data.append(best_dupkey)
        decision_dict[dupkey] = 'Valid_msms'
        continue
    
    else:
        
        if sum(dupkey_df.notnull()['metatlas_score'].values) > 0: # If metatlas score exists, use the highest one
                
            if len(dupkey_df['metatlas_score'].unique()) != 1:

                dupkey_df.sort_values('metatlas_score', ascending = False)
                best_dupkey = dupkey_df.head(1)

                print(colored('Filtered: ', 'green') + 'Metatlas score\n')
                deduplicated_data.append(best_dupkey)
                decision_dict[dupkey] = 'Metatlas_score'
                continue

        if "Platinum" in str(dupkey_df['top_two_by_rank'].value_counts().index) and dupkey_df['top_two_by_rank'].value_counts()['Platinum ' + polarity + ' max rank'] == 1: # Only one row with platinum confidence

            best_dupkey = dupkey_df[dupkey_df['top_two_by_rank'] == 'Platinum ' + polarity + ' max rank']

            print(colored('Filtered: ', 'green') + 'Confidence category\n')
            deduplicated_data.append(best_dupkey)
            decision_dict[dupkey] = 'Platinum_confidence'
            continue

        if len(dupkey_df['code_rank_without_msms'].unique()) != 1: # If code rank exists, use the best one

            dupkey_df.sort_values('code_rank_without_msms', ascending = True)
            best_dupkey = dupkey_df.head(1)

            print(colored('Filtered: ', 'green') + 'Code rank\n')
            deduplicated_data.append(best_dupkey)
            decision_dict[dupkey] = 'Code_rank_integer'
            continue

        if sum(dupkey_df.notnull()['nist_score'].values) > 0: # If NIST score exists, use the highest one
                
            if len(dupkey_df['nist_score'].unique()) != 1:
                
                dupkey_df.sort_values('nist_score', ascending = False)
                best_dupkey = dupkey_df.head(1)

                print(colored('Filtered: ', 'green') + 'NIST score\n')
                deduplicated_data.append(best_dupkey)
                decision_dict[dupkey] = 'NIST_score'
                continue
            
        if sum(dupkey_df.notnull()['msms_score'].values) > 0: # If MSMS score exists, use the highest one
                
            if len(dupkey_df['msms_score'].unique()) != 1:
                
                dupkey_df.sort_values('msms_score', ascending = False)
                best_dupkey = dupkey_df.head(1)

                print(colored('Filtered: ', 'green') + 'MSMS score\n')
                deduplicated_data.append(best_dupkey)
                decision_dict[dupkey] = 'MSMS_score'
                continue

        if sum(dupkey_df['peak_height_ms1-summary_205060'].values) + sum(dupkey_df['peak_height_ms1-summary_102040'].values) > 0: # Choose entry with largest peak heights in both energies
            
            sorted_indices = (dupkey_df["peak_height_ms1-summary_102040"] + dupkey_df["peak_height_ms1-summary_205060"]).sort_values().index
            dupkey_df.loc[sorted_indices, :]
            best_dupkey = dupkey_df.head(1)

            print(colored('Filtered: ', 'green') + 'Peak heights\n')
            deduplicated_data.append(best_dupkey)
            decision_dict[dupkey] = 'Peak_heights'
            continue

        if "multiple peaks" in str(dupkey_df['Peak shape'].value_counts().index) and dupkey_df['Peak shape'].value_counts()['multiple peaks'] == dupkey_df.shape[0]-1: # If there is only one row without 'multiple peaks', use it
            
            best_dupkey = dupkey_df[dupkey_df['Peak shape'] != 'multiple peaks']

            print("Filtered by peak shape!")
            deduplicated_data.append(best_dupkey)
            decision_dict[dupkey] = 'Peak_shape'
            
        else:

            print(colored('Warning: ', 'red') + 'Unresolved decision tree\n')
            decision_dict[dupkey] = 'Unresolved'

decision_list = list(decision_dict.values())
{i:decision_list.count(i) for i in set(decision_list)}

In [216]:
# Combine clean file without duplicates and the deduplicated df

deduplicated_df = pd.concat(deduplicated_data)
original_and_updated_merge_no_duplicates = original_and_updated_merge[~original_and_updated_merge['inchi-adduct'].isin(duplicated_inchis)]

new_deduplicated_standards = pd.concat([deduplicated_df, original_and_updated_merge_no_duplicates], axis = 0)

In [217]:
# Write new deduplicated df

new_deduplicated_standards.to_csv('C18_standards_' + polarity + '_all-adducts_renamed_deduplicated.tsv', sep='\t', index = False)